# Unstructured data pipeline for the Agent's Retriever

By the end of this notebook, you will have transformed your unstructured documents into a vector index that can be queried by your Agent.

This means:
- Documents loaded into a delta table.
- Documents are chunked.
- Chunks have been embedded with an embedding model and stored in a vector index.

The important resulting artifact of this notebook is the chunked vector index. This will be used in the next notebook to power our Retriever.

### 👉 START HERE: How to Use This Notebook

Follow these steps to build and refine your data pipeline's quality:

1. **Build a v0 index with default settings**
    - Configure the data source and destination tables in the `1️⃣ 📂 Data source & destination configuration` cells
    - Press `Run All` to create the vector index.

    *Note: While you can adjust the other settings and modify the parsing/chunking code, we suggest doing so only after evaluating your Agent's quality so you can make improvements that specifically address root causes of quality issues.*

2. **Use later notebooks to integrate the retriever into an the agent and evaluate the agent/retriever's quality.**

3. **If the evaluation results show retrieval issues as a root cause, use this notebook to iterate on your data pipeline's code & config.** Below are some potential fixes you can try, see the AI Cookbook's [debugging retrieval issues](https://ai-cookbook.io/nbs/5-hands-on-improve-quality-step-1-retrieval.html) section for details.**
    - Add missing, but relevant source documents into in the index.
    - Resolve any conflicting information in source documents.
    - Adjust the data pipeline configuration:
      - Modify chunk size or overlap.
      - Experiment with different embedding models.
    - Adjust the data pipeline code:
      - Create a custom parser or use different parsing libraries.
      - Develop a custom chunker or use different chunking techniques.
      - Extract additional metadata for each document.
    - Adjust the Agent's code/config in subsequent notebooks:
      - Change the number of documents retrieved (K).
      - Try a re-ranker.
      - Use hybrid search.
      - Apply extracted metadata as filters.



**Important note:** Throughout this notebook, we indicate which cells you:
- ✅✏️ *should* customize - these cells contain code & config with business logic that you should edit to meet your requirements & tune quality
- 🚫✏️ *typically will not* customize - these cells contain boilerplate code required to execute the pipeline

*Cells that don't require customization still need to be run!  You CAN change these cells, but if this is the first time using this notebook, we suggest not doing so.*

### Install Python libraries

🚫✏️ Only modify if you need additional packages in your code changes to the document parsing or chunking logic.

Versions of Databricks code are not locked since Databricks ensures changes are backwards compatible.
Versions of open source packages are locked since package authors often make backwards compatible changes

In [ ]:
%pip install -qqqq -U -r requirements.txt
%pip install -qqqq -U -r requirements_datapipeline.txt
dbutils.library.restartPython()

If running from an IDE with [`databricks-connect`](https://docs.databricks.com/en/dev-tools/databricks-connect/python/index.html), connect to a Spark session.

In [ ]:
from utils.cookbook.databricks_utils import get_active_cluster_id, get_cluster_url
from utils.cookbook.install_cluster_library import install_requirements
# cluster_id = get_active_cluster_id()
# print(f"Installing packages on the active cluster: {get_cluster_url(cluster_id)}")

# # TODO: build the utils wheel and install it 
# install_requirements(cluster_id, "requirements.txt")
# install_requirements(cluster_id, "requirements_datapipeline.txt")

# Get Spark session if using Databricks Connect from an IDE
from mlflow.utils import databricks_utils as du

if not du.is_in_databricks_notebook():
    from databricks.connect import DatabricksSession

    spark = DatabricksSession.builder.getOrCreate()


## 1️⃣ 📂 Data source & destination configuration

#### ✅✏️ Configure the data pipeline's source location.

Choose a [Unity Catalog Volume](https://docs.databricks.com/en/volumes/index.html) containing PDF, HTML, etc documents to be parsed/chunked/embedded.

- `uc_catalog_name`: Name of the Unity Catalog.
- `uc_schema_name`: Name of the Unity Catalog schema.
- `uc_volume_name`: Name of the Unity Catalog volume.

Running this cell with validate that the UC Volume exists, trying to create it if not.


In [ ]:
from utils.data_pipeline.data_pipeline_config import UCVolumeSourceConfig

# Configure the UC Volume that contains the source documents
source_config = UCVolumeSourceConfig(
    uc_catalog_name="REPLACE_ME", # REPLACE_ME
    uc_schema_name="REPLACE_ME", # REPLACE_ME
    uc_volume_name=f"REPLACE_ME", # REPLACE_ME
)

# Check if volume exists, create otherwise
is_valid, msg = source_config.create_or_validate_volume()
if not is_valid:
    raise Exception(msg)

#### ✅✏️ Configure the data pipeline's output location.
 
Choose where the data pipeline outputs the parsed, chunked, and embedded documents.

Required parameters:
* `uc_catalog_name`: Unity Catalog name where tables will be created
* `uc_schema_name`: Schema name within the catalog 
* `base_table_name`: Core name used as prefix for all generated tables
* `vector_search_endpoint`: Vector Search endpoint to store the index

Optional parameters:
* `docs_table_postfix`: Suffix for the parsed documents table (default: "docs")
* `chunked_table_postfix`: Suffix for the chunked documents table (default: "docs_chunked") 
* `vector_index_postfix`: Suffix for the vector index (default: "docs_chunked_index")
* `version_suffix`: Version identifier (e.g. 'v1', 'test') to maintain multiple versions

The generated tables follow this naming convention:
* Parsed docs: {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{docs_table_postfix}__{version_suffix}
* Chunked docs: {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{chunked_table_postfix}__{version_suffix}
* Vector index: {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{vector_index_postfix}__{version_suffix}

*Note: If you are comparing different chunking/parsing/embedding strategies, set the `version_suffix` parameter to maintain multiple versions of the pipeline output with the same base_table_name.*

*Databricks suggests sharing a Vector Search endpoint across multiple agents.*

In [ ]:
from utils.data_pipeline.data_pipeline_config import DataPipelineOuputConfig

# Output configuration
output_config = DataPipelineOuputConfig(
    # Required parameters
    uc_catalog_name="REPLACE_ME", # usually same as source volume catalog
    uc_schema_name="REPLACE_ME", # usually same as source volume schema
    base_table_name="REPLACE_ME", # usually similar / same as the source volume name
    vector_search_endpoint="REPLACE_ME", # Vector Search endpoint to store the index

    # Optional parameters, showing defaults
    docs_table_postfix="docs",              # default value is `docs`
    chunked_table_postfix="docs_chunked",   # default value is `docs_chunked`
    vector_index_postfix="docs_chunked_index", # default value is `docs_chunked_index`
    version_suffix="v1"                     # default is None

    # Output tables / indexes follow this naming convention:
    # {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{docs_table_postfix}__{version_suffix}
    # {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{chunked_table_postfix}__{version_suffix}
    # {uc_catalog_name}.{uc_schema_name}.{base_table_name}_{vector_index_postfix}__{version_suffix}
)

# Check UC locations exist
is_valid, msg = output_config.validate_catalog_and_schema()
if not is_valid:
    raise Exception(msg)

# Check Vector Search endpoint exists
is_valid, msg = output_config.validate_vector_search_endpoint()
if not is_valid:
    raise Exception(msg)

#### ✅✏️ Configure chunk size and the embedding model.

**Chunk size and overlap** control how a larger document is turned into smaller chunks that can be processed by an embedding model.  See the AI Cookbook [chunking deep dive](https://ai-cookbook.io/nbs/3-deep-dive-data-pipeline.html#chunking) for more details.

**The embedding model** is an AI model that is used to identify the most similar documents to a given user's query.  See the AI Cookbook [embedding model deep dive](https://ai-cookbook.io/nbs/3-deep-dive-data-pipeline.html#embedding-model) for more details.

This notebook supports the following [Foundational Models](https://docs.databricks.com/en/machine-learning/foundation-models/index.html) or [External Model](https://docs.databricks.com/en/generative-ai/external-models/index.html) of type `/llm/v1/embeddings`/.  If you want to try another model, you will need to modify the `utils/get_recursive_character_text_splitter` Notebook to add support.
- `databricks-gte-large-en` or `databricks-bge-large-en`
- Azure OpenAI or OpenAI External Model of type `text-embedding-ada-002`, `text-embedding-3-small` or `text-embedding-3-large`

In [ ]:
from utils.data_pipeline.recursive_character_text_splitter import ChunkingConfig

chunking_config = ChunkingConfig(
    embedding_model_endpoint="databricks-gte-large-en",  # A Model Serving endpoint supporting the /llm/v1/embeddings task
    chunk_size_tokens=1024,
    chunk_overlap_tokens=256,
)

# Validate the embedding endpoint & chunking config
is_valid, msg = chunking_config.validate_embedding_endpoint()
if not is_valid:
    raise Exception(msg)

is_valid, msg = chunking_config.validate_chunk_size_and_overlap()
if not is_valid:
    raise Exception(msg)

#### 🚫✏️ Write the data pipeline configuration to a YAML

This allows the configuration to be loaded referenced by the Agent's notebook.

In [4]:
from utils.data_pipeline.data_pipeline_config import DataPipelineConfig
from utils.agents.tools import obj_to_yaml_file

data_pipeline_config = DataPipelineConfig(
    source=source_config,
    output=output_config,
    chunking_config=chunking_config,
)

obj_to_yaml_file(data_pipeline_config, "./configs/data_pipeline_config.yaml")

#### 🛑 If you are running your initial data pipeline, you do not need to configure anything else, you can just `Run All` the notebook cells before.  You can modify these cells later to tune the quality of your data pipeline by changing the parsing logic.

## 3️⃣ ⌨️ Data pipeline code

The code below executes the data pipeline.  You can modify the below code as indicated to implement different parsing or chunking strategies or to extract additional metadata fields

#### Pipeline step 1: Load & parse documents into a Delta Table

In this step, we'll load files from the UC Volume defined in `source_config` into the Delta Table `storage_config.parsed_docs_table` . The contents of each file will become a separate row in our delta table.

The path to the source document will be used as the `doc_uri` which is displayed to your end users in the Agent Evalution web application.

After you test your POC with stakeholders, you can return here to change the parsing logic or extraction additional metadata about the documents to help improve the quality of your retriever.

##### ✅✏️ Customize the parsing function

This default implementation parses PDF, HTML, and DOCX files using open source libraries.  Adjust `file_parser(...)` to add change the parsing logic, add support for more file types, or extract additional metadata about each document.

In [9]:
from typing import TypedDict
from datetime import datetime
import warnings
import traceback
import os
from urllib.parse import urlparse

# PDF libraries
import fitz
import pymupdf4llm

# HTML libraries
import markdownify
import re

## DOCX libraries
import pypandoc
import tempfile

## JSON libraries
import json


# Schema of the dict returned by `file_parser(...)`
# This is used to create the output Delta Table's schema.
# Adjust the class if you want to add additional columns from your parser, such as extracting custom metadata.
class ParserReturnValue(TypedDict):
    # DO NOT CHANGE THESE NAMES
    # Parsed content of the document
    content: str # do not change this name
    # The status of whether the parser succeeds or fails, used to exclude failed files downstream
    parser_status: str  # do not change this name
    # Unique ID of the document
    doc_uri: str  # do not change this name

    # OK TO CHANGE THESE NAMES
    # Optionally, you can add additional metadata fields here
    # example_metadata: str
    last_modified: datetime


# Parser function.  Adjust this function to modify the parsing logic.
def file_parser(
    raw_doc_contents_bytes: bytes,
    doc_path: str,
    modification_time: datetime,
    doc_bytes_length: int,
) -> ParserReturnValue:
    """
    Parses the content of a PDF document into a string.

    This function takes the raw bytes of a PDF document and its path, attempts to parse the document using PyPDF,
    and returns the parsed content and the status of the parsing operation.

    Parameters:
    - raw_doc_contents_bytes (bytes): The raw bytes of the document to be parsed (set by Spark when loading the file)
    - doc_path (str): The DBFS path of the document, used to verify the file extension (set by Spark when loading the file)
    - modification_time (timestamp): The last modification time of the document (set by Spark when loading the file)
    - doc_bytes_length (long): The size of the document in bytes (set by Spark when loading the file)

    Returns:
    - ParserReturnValue: A dictionary containing the parsed document content and the status of the parsing operation.
      The 'contenty will contain the parsed text as a string, and the 'parser_status' key will indicate
      whether the parsing was successful or if an error occurred.
    """
    try:
        from markdownify import markdownify as md

        filename, file_extension = os.path.splitext(doc_path)

        if file_extension == ".pdf":
            pdf_doc = fitz.Document(stream=raw_doc_contents_bytes, filetype="pdf")
            md_text = pymupdf4llm.to_markdown(pdf_doc)

            parsed_document = {
                "content": md_text.strip(),
                "parser_status": "SUCCESS",
            }
        elif file_extension == ".html":
            html_content = raw_doc_contents_bytes.decode("utf-8")

            markdown_contents = md(
                str(html_content).strip(), heading_style=markdownify.ATX
            )
            markdown_stripped = re.sub(r"\n{3,}", "\n\n", markdown_contents.strip())

            parsed_document = {
                "content": markdown_stripped,
                "parser_status": "SUCCESS",
            }
        elif file_extension == ".docx":
            with tempfile.NamedTemporaryFile(delete=True) as temp_file:
                temp_file.write(raw_doc_contents_bytes)
                temp_file_path = temp_file.name
                md = pypandoc.convert_file(temp_file_path, "markdown", format="docx")

                parsed_document = {
                    "content": md.strip(),
                    "parser_status": "SUCCESS",
                }
        elif file_extension in [".txt", ".md"]:
            parsed_document = {
                "content": raw_doc_contents_bytes.decode("utf-8").strip(),
                "parser_status": "SUCCESS",
            }
        elif file_extension in [".json", ".jsonl"]:
            # NOTE: This is a placeholder for a JSON parser.  It's not a "real" parser, it just returns the raw JSON formatted into XML-like strings that LLMs tend to like.
            json_data = json.loads(raw_doc_contents_bytes.decode("utf-8"))
            
            def flatten_json_to_xml(obj, parent_key=''):
                xml_parts = []
                if isinstance(obj, dict):
                    for key, value in obj.items():
                        if isinstance(value, (dict, list)):
                            xml_parts.append(flatten_json_to_xml(value, key))
                        else:
                            xml_parts.append(f"<{key}>{str(value)}</{key}>")
                elif isinstance(obj, list):
                    for i, item in enumerate(obj):
                        if isinstance(item, (dict, list)):
                            xml_parts.append(flatten_json_to_xml(item, f"{parent_key}_{i}"))
                        else:
                            xml_parts.append(f"<{parent_key}_{i}>{str(item)}</{parent_key}_{i}>")
                else:
                    xml_parts.append(f"<{parent_key}>{str(obj)}</{parent_key}>")
                return "\n".join(xml_parts)
                
            flattened_content = flatten_json_to_xml(json_data)
            parsed_document = {
                "content": flattened_content.strip(),
                "parser_status": "SUCCESS",
            }
        else:
            raise Exception(f"No supported parser for {doc_path}")

        # Extract the required doc_uri
        # convert from `dbfs:/Volumes/catalog/schema/pdf_docs/filename.pdf` to `/Volumes/catalog/schema/pdf_docs/filename.pdf`
        modified_path = urlparse(doc_path).path
        parsed_document["doc_uri"] = modified_path

        # Sample metadata extraction logic
        # if "test" in parsed_document["content
        #     parsed_document["example_metadata"] = "test"
        # else:
        #     parsed_document["example_metadata"] = "not test"

        # Add the modified time
        parsed_document["last_modified"] = modification_time

        return parsed_document

    except Exception as e:
        status = f"An error occurred: {e}\n{traceback.format_exc()}"
        warnings.warn(status)
        return {
            "content": "",
            "parser_status": f"ERROR: {status}",
        }


The below cell is debugging code to test your parsing function on a single record. 

In [ ]:
from utils.data_pipeline.file_loading import load_files_to_df
from pyspark.sql import functions as F


raw_files_df = load_files_to_df(
    spark=spark,
    source_path=source_config.volume_path,
)

print(f"Loaded {raw_files_df.count()} files from {source_config.volume_path}.  Files: {source_config.list_files()}")

test_records_dict = raw_files_df.toPandas().to_dict(orient="records")

for record in test_records_dict:
  print()
  print("Testing parsing for file: ", record["path"])
  print()
  test_result = file_parser(raw_doc_contents_bytes=record['content'], doc_path=record['path'], modification_time=record['modificationTime'], doc_bytes_length=record['length'])
  print(test_result)
  break # pause after 1 file.  if you want to test more files, remove the break statement


🚫✏️ The below cell is boilerplate code to apply the parsing function using Spark.

In [ ]:
from utils.data_pipeline.file_loading import (
    load_files_to_df,
    apply_parsing_fn,
    check_parsed_df_for_errors,
    check_parsed_df_for_empty_parsed_files
)
from utils.data_pipeline.typed_dicts_to_spark_schema import typed_dicts_to_spark_schema
from pyspark.errors import PySparkAttributeError

# Load the UC Volume files into a Spark DataFrame
raw_files_df = load_files_to_df(
    spark=spark,
    source_path=source_config.volume_path,
)

# Apply the parsing UDF to the Spark DataFrame
parsed_files_df = apply_parsing_fn(
    raw_files_df=raw_files_df,
    # Modify this function to change the parser, extract additional metadata, etc
    parse_file_fn=file_parser,
    # The schema of the resulting Delta Table will follow the schema defined in ParserReturnValue
    parsed_df_schema=typed_dicts_to_spark_schema(ParserReturnValue),
)

# Write to a Delta Table
parsed_files_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    output_config.parsed_docs_table
)

# Get resulting table
parsed_files_df = spark.table(output_config.parsed_docs_table)

print(f"Parsed {parsed_files_df.count()} documents: ")
try:
    parsed_files_df.display()
except PySparkAttributeError as e:
    parsed_files_df.show()

Check parsing results for any errors or empty parsing results.

In [ ]:

# Any documents that failed to parse
is_error, msg, failed_docs_df = check_parsed_df_for_errors(parsed_files_df)
if is_error:
    try: # if in DBX notebook, render the dataframe, otherwise print it
        failed_docs_df.display()
    except PySparkAttributeError as e:
        failed_docs_df.show()
    raise Exception(msg)
    
# Any documents that returned empty parsing results
is_error, msg, empty_docs_df = check_parsed_df_for_empty_parsed_files(parsed_files_df)
if is_error:
    try: # if in DBX notebook, render the dataframe, otherwise print it
        empty_docs_df.display()
    except PySparkAttributeError as e:
        empty_docs_df.show()
    raise Exception(msg)
    

# Filter for records without errors
parsed_files_no_errors_df = parsed_files_df.filter(
    parsed_files_df.parser_status == "SUCCESS"
)

# Display for results debugging
print(f"Parsed {parsed_files_no_errors_df.count()} documents: ")
try:
    parsed_files_no_errors_df.display()
except PySparkAttributeError as e:
    parsed_files_no_errors_df.show()

#### Pipeline step 2: Compute chunks of documents

In this step, we will split our documents into smaller chunks so they can be indexed in our vector database.


##### ✅✏️ Chunking logic.

We provide a default implementation of a recursive text splitter.  To create your own chunking logic, adapt the `get_recursive_character_text_splitter()` function inside `cookbook_utils.recursive_character_text_splitter.py`.

In [ ]:
from utils.data_pipeline.recursive_character_text_splitter import (
    get_recursive_character_text_splitter,
)

# Get the chunking function
recursive_character_text_splitter_fn = get_recursive_character_text_splitter(
    model_serving_endpoint=chunking_config.embedding_model_endpoint,
    chunk_size_tokens=chunking_config.chunk_size_tokens,
    chunk_overlap_tokens=chunking_config.chunk_overlap_tokens,
)

# Determine which columns to propagate from the docs table to the chunks table.

# Get the columns from the parser except for the content
# You can modify this to adjust which fields are propagated from the docs table to the chunks table.
propagate_columns = [
    field.name
    for field in typed_dicts_to_spark_schema(ParserReturnValue).fields
    if field.name != "content"
]

# If you want to implement retrieval strategies such as presenting the entire document vs. the chunk to the LLM, include `contentich contains the doc's full parsed text.  By default this is not included because the size of contcontentquite large and cause performance issues.
# propagate_columns = [
#     field.name
#     for field in typed_dicts_to_spark_schema(ParserReturnValue).fields
# ]

🚫✏️ Run the chunking function within Spark

In [ ]:
from utils.data_pipeline.chunk_docs import apply_chunking_fn

parsed_files_df = spark.table(output_config.parsed_docs_table)

chunked_docs_df = chunked_docs_table = apply_chunking_fn(
    # The source documents table.
    parsed_docs_df=parsed_files_df,
    # The chunking function that takes a string (document) and returns a list of strings (chunks).
    chunking_fn=recursive_character_text_splitter_fn,
    # Choose which columns to propagate from the docs table to chunks table. `doc_uri` column is required we can propagate the original document URL to the Agent's web app.
    propagate_columns=propagate_columns,
)

# Write to Delta Table
chunked_docs_df.write.mode("overwrite").option(
    "overwriteSchema", "true"
).saveAsTable(output_config.chunked_docs_table)

# Get resulting table
chunked_docs_df = spark.table(output_config.chunked_docs_table)

# enable CDC feed for VS index sync
cdc_results = spark.sql(f"ALTER TABLE {output_config.chunked_docs_table} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

try: # if in DBX notebook, render the dataframe, otherwise print it
    chunked_docs_df.display()
except PySparkAttributeError as e:
    chunked_docs_df.show()

#### 🚫✏️ Pipeline step 3: Create the vector index

In this step, we'll embed the documents to compute the vector index over the chunks and create our retriever index that will be used to query relevant documents to the user question.  The embedding pipeline is handled within Databricks Vector Search using [Delta Sync](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html#create-a-vector-search-index)

In [ ]:
from utils.data_pipeline.build_retriever_index import build_retriever_index
from utils.cookbook.databricks_utils import get_table_url

is_error, msg = retriever_index_result = build_retriever_index(
    # Spark requires `` to escape names with special chars, VS client does not.
    chunked_docs_table_name=output_config.chunked_docs_table.replace("`", ""),
    vector_search_endpoint=output_config.vector_search_endpoint,
    vector_search_index_name=output_config.vector_index,

    # Must match the embedding endpoint you used to chunk your documents
    embedding_endpoint_name=chunking_config.embedding_model_endpoint,

    # Set to true to re-create the vector search endpoint when re-running the data pipeline.  If set to True, syncing will not work if re-run the pipeline and change the schema of chunked_docs_table_name.  Keeping this as False will allow Vector Search to avoid recomputing embeddings for any row with that has a chunk_id that was previously computed.
    force_delete_index_before_create=False,
)
if is_error:
    raise Exception(msg)
else:
    print(f"View sync status here: {get_table_url(output_config.vector_index)}")


#### 🚫✏️ Print links to view the resulting tables/index

In [ ]:
from utils.cookbook.databricks_utils import get_table_url

print()
print(f"Parsed docs table: {get_table_url(output_config.parsed_docs_table)}\n")
print(f"Chunked docs table: {get_table_url(output_config.chunked_docs_table)}\n")
print(f"Vector search index: {get_table_url(output_config.vector_index)}\n")